In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.cm import get_cmap
import seaborn as sns
import string
import pickle # save data frame (results) in a .pkl file
import pandas as pd
from datetime import datetime
import os, sys
import re

import grakel as gk


sys.path.insert(0, 'C:/Users/ragna/Documents/Ragnar/Code/MMDGraph')
sys.path.insert(0, 'C:/Users/ragna/Documents/Ragnar/Code/MMDGraph/myKernels')
import mmdutils
import importlib
importlib.reload(sys.modules['mmdutils'])
import MMDforGraphs as mg
importlib.reload(sys.modules['MMDforGraphs'])
import RandomWalk as rw


In [11]:
from importlib import reload  
foo = reload(mg)

n_1 = 50
n_2 = 50
nnode_1 = 140
nnode_2 = 140


bg1 = mg.BinomialGraphs(n_1, nnode_1, k = 4, l = 'degreelabels', fullyConnected = True)
bg2 = mg.BinomialGraphs(n_2, nnode_2, k = 4.25, l = 'degreelabels', fullyConnected = True)

bg1.Generate()
bg2.Generate()
Gs = bg1.Gs + bg2.Gs

ARKU (allows weights and edge directions) approximation

In [4]:

importlib.reload(sys.modules['RandomWalk'])
rw_kernel = rw.RandomWalk(Gs, 0.1)
K = rw_kernel.fit_ARKU(r = 6)


100%|██████████| 820/820.0 [00:16<00:00, 48.57it/s]


ARKU plus faster

In [5]:
importlib.reload(sys.modules['RandomWalk'])
rw_kernel = rw.RandomWalk(Gs, 0.1)
K = rw_kernel.fit_ARKU_plus(r = 6, normalize_adj=True)

100%|██████████| 820/820.0 [00:06<00:00, 119.82it/s]


 ARK-L, allow labels

First we have to fetch all labels generated, usually this should be known

In [6]:
label_list = []
for G in Gs:
    label_list.append(np.unique(list(nx.get_node_attributes(G, 'label').values())))

label_list = np.unique(np.concatenate(label_list))


In [7]:
importlib.reload(sys.modules['RandomWalk'])
rw_kernel = rw.RandomWalk(Gs, 0.1)
K = rw_kernel.fit_ARKL(r = 6, label_list = label_list, normalize_adj=True)

100%|██████████| 820/820.0 [00:07<00:00, 114.88it/s]


Fit random walk with k walks, no approximation is going to be slow.

In [9]:
importlib.reload(sys.modules['RandomWalk'])
rw_kernel = rw.RandomWalk(Gs, 0.1)
mu_vec = np.power(0.1,range(6+1)) / np.array([np.math.factorial(i) for i in np.arange(6+1)])
K = rw_kernel.fit_random_walk(mu_vec, k = 6,  r = 10, normalize_adj=True )

100%|██████████| 820/820.0 [00:08<00:00, 96.74it/s] 


The following is the grakel package, using kernel type:exponential is slow.

In [53]:
kernel = [{"name":'RW', 'kernel_type':'geometric', 'lamda':0.1, 'with_labels':False}]
# kernel = [{"name":'odd_sth', 'h':3}]
# kernel = [{"name": "propagation", "t_max": 4, "w":1, "M":'TV'}]
init_kernel = gk.GraphKernel(kernel= kernel, normalize=False, n_jobs= 1)
K = init_kernel.fit_transform(gk.graph_from_networkx(Gs, node_labels_tag='label')  )


In [15]:
K

array([[5.37391717e-05, 5.29583871e-05, 5.38485706e-05, ...,
        5.36500964e-05, 5.43479803e-05, 5.30179344e-05],
       [5.29583871e-05, 5.24054302e-05, 5.30353136e-05, ...,
        5.28956895e-05, 5.33850717e-05, 5.24493123e-05],
       [5.38485706e-05, 5.30353136e-05, 5.39626855e-05, ...,
        5.37556745e-05, 5.44840505e-05, 5.30968309e-05],
       ...,
       [5.36500964e-05, 5.28956895e-05, 5.37556745e-05, ...,
        5.35641176e-05, 5.42373217e-05, 5.29536069e-05],
       [5.43479803e-05, 5.33850717e-05, 5.44840505e-05, ...,
        5.42373217e-05, 5.51081766e-05, 5.34550053e-05],
       [5.30179344e-05, 5.24493123e-05, 5.30968309e-05, ...,
        5.29536069e-05, 5.34550053e-05, 5.24950776e-05]])

In [10]:

MMD_functions = [mg.MMD_b, mg.MMD_u]

# initialize bootstrap class, we only want this to be initalized once so that numba njit
# only gets compiled once (at first call)
kernel_hypothesis = mg.BoostrapMethods(MMD_functions)
function_arguments=[dict(n = bg1.n, m = bg2.n ), dict(n = bg1.n, m = bg2.n )]
kernel_hypothesis.Bootstrap(K, function_arguments, B = 1000)
print(kernel_hypothesis.p_values)

{'MMD_b': 0.738, 'MMD_u': 0.738}
